# Melting and pivoting data 
With some display examples for .Net Interactive notebooks. 


## Setup the data

In [ ]:
Import-Module ..\PowerShellPivot.psd1 -Force
$data = ConvertFrom-Csv @"
Day,Location,Cucumber,Tomato,Lettuce,Asparagus,Potato
Monday,London,46,35,41,49,30
Tuesday,London,30,26,36,38,27
Wednesday,London,25,26,27,31,30
Thursday,London,47,32,44,21,37
Friday,London,38,40,35,27,39
Saturday,London,32,29,39,32,31
Sunday,London,28,31,37,29,39
Monday,Edinburgh,29,26,36,25,35
Tuesday,Edinburgh,44,48,32,26,28
Wednesday,Edinburgh,46,43,38,41,26
Thursday,Edinburgh,39,39,36,31,20
Friday,Edinburgh,36,38,47,30,24
Saturday,Edinburgh,33,27,39,47,39
Sunday,Edinburgh,43,42,35,37,28
Monday,Glasgow,29,30,25,29,47
Tuesday,Glasgow,32,36,46,38,22
Wednesday,Glasgow,20,33,26,44,27
Thursday,Glasgow,21,29,25,35,46
Friday,Glasgow,39,36,45,28,32
Saturday,Glasgow,22,34,33,33,29
Sunday,Glasgow,27,23,29,24,24
Monday,Birmingham,27,34,49,35,31
Tuesday,Birmingham,39,41,41,31,20
Wednesday,Birmingham,33,46,28,40,47
Thursday,Birmingham,21,46,38,29,20
Friday,Birmingham,31,32,21,32,31
Saturday,Birmingham,44,20,46,26,29
Sunday,Birmingham,47,41,42,23,29
Monday,Cardiff,34,37,40,30,41
Tuesday,Cardiff,47,41,26,24,49
Wednesday,Cardiff,27,40,38,26,32
Thursday,Cardiff,29,28,43,26,22
Friday,Cardiff,21,43,23,37,27
Saturday,Cardiff,42,44,47,38,47
Sunday,Cardiff,37,31,21,28,26
"@

## A random sample of the data looks like...

In [ ]:
$data |  Get-Random -Count 10 | Format-Table


Day       Location   Cucumber Tomato Lettuce Asparagus Potato
---       --------   -------- ------ ------- --------- ------
Friday    Glasgow    39       36     45      28        32
Friday    Birmingham 31       32     21      32        31
Wednesday Glasgow    20       33     26      44        27
Thursday  Glasgow    21       29     25      35        46
Saturday  Edinburgh  33       27     39      47        39
Friday    London     38       40     35      27        39
Friday    Edinburgh  36       38     47      30        24
Saturday  Birmingham 44       20     46      26        29
Sunday    Edinburgh  43       42     35      37        28
Monday    Glasgow    29       30     25      29        47



## This data is shown in 2 dimensions for human reading...

We have the products as a column headings, we've grouped the data by location and then by day - as a report we might have page breaks between locations. 
We can add totals to each column (product), and give subtotals per location as in the next cell (or change the sort order and have subtotals per day as in the cell after that)

In [ ]:
$data | New-PSPivotTable -Index Location -ExcludeValues Location | Format-Table 


Location   Average_Cucumber Average_Tomato Average_Lettuce Average_Asparagus Average_Potato
--------   ---------------- -------------- --------------- ----------------- --------------
Birmingham           34.571         37.143          37.857            30.857         29.571
Cardiff              33.857         37.714              34            29.857         34.857
Edinburgh            38.571         37.571          37.571            33.857         28.571
Glasgow              27.143         31.571          32.714                33         32.429
London               35.143         31.286              37            32.429         33.286



Columns  must be used as a single column with a single index, a single value and a single aggregate function


In [ ]:
$data | New-PSPivotTable -Index Day -ExcludeValues Location | Format-Table


Day       Average_Cucumber Average_Tomato Average_Lettuce Average_Asparagus Average_Potato
---       ---------------- -------------- --------------- ----------------- --------------
Friday                  33           37.8            34.2              30.8           30.6
Monday                  33           32.4            38.2              33.6           36.8
Saturday              34.6           30.8            40.8              35.2             35
Sunday                36.4           33.6            32.8              28.2           29.2
Thursday              31.4           34.8            37.2              28.4             29
Tuesday               38.4           38.4            36.2              31.4           29.2
Wednesday             30.2           37.6            31.4              36.4           32.4



What we can't do with the data like this is change from a page per location with products as column headings, to one with a page per product and locations as column headings. 

## Combining \[product\] columns can be an issue... 
If we know what the columns (products) are we can add have a formula to calculate Lettuce + Cucumber + Tomato,    
if not, it helps to "melt" this data getting it back from the 2D presentation to a more a ; And if one location files as days down and products across and another    
products down and days across we can merge them by melting first. 

**The next cell melts data and both stores it and shows a random selection**

In [ ]:
#melt the data and keep the result in 'DataToPivot' but output a few random rws as well
$data | Invoke-PSMelt -Id Day, location -ValueName Sales -VarName Product -OutVariable DataToPivot  | Get-Random -Count 10 | ft


Day       location   Product   Sales
---       --------   -------   -----
Wednesday Glasgow    Lettuce   26
Monday    Birmingham Asparagus 35
Thursday  Edinburgh  Asparagus 31
Sunday    London     Asparagus 29
Saturday  Edinburgh  Asparagus 47
Saturday  Edinburgh  Tomato    27
Friday    Edinburgh  Tomato    38
Monday    Edinburgh  Asparagus 25
Saturday  Birmingham Cucumber  44
Tuesday   Glasgow    Tomato    36



## 'Melted Data' is easier to work with

In [ ]:
$dataToPivot | where {$_.product -in @('Lettuce','Cucumber','Tomato')} | New-PSPivotTable Day -Agg Average, Min, Max 


Day       Average_Sales Min_Sales Max_Sales
---       ------------- --------- ---------
Friday               35        21        47
Monday           34.533        25        49
Saturday           35.4        20        47
Sunday           34.267        21        47
Thursday         34.467        21        47
Tuesday          37.667        26        48
Wednesday        33.067        20        46



In [ ]:
#parameters for a function later , we already have $DataToPivot from the previous cell 
$PivotBy     = 'Day' 
$aggFunction = 'Average', 'Min', 'Max' 

$pivoted     = $dataToPivot | where {$_.product -in @('Lettuce','Cucumber','Tomato')} | New-PSPivotTable $PivotBy -agg $aggFunction  
$pivoted     = foreach ($day in (Get-Culture).DateTimeFormat.DayNames) {$pivoted | Where-Object -Property Day -eq $day}
$pivoted



Day       Average_Sales Min_Sales Max_Sales
---       ------------- --------- ---------
Sunday           34.267        21        47
Monday           34.533        25        49
Tuesday          37.667        26        48
Wednesday        33.067        20        46
Thursday         34.467        21        47
Friday               35        21        47
Saturday           35.4        20        47



## Alternative outputs

In [ ]:
# load out-cell and out-tree view
. ..\NotebookOutput.ps1
$pivoted | Out-cell -AsTable


Day,Average_Sales,Min_Sales,Max_Sales
Sunday,34.267,21,47
Monday,34.533,25,49
Tuesday,37.667,26,48
Wednesday,33.067,20,46
Thursday,34.467,21,47
Friday,35,21,47
Saturday,35.4,20,47


In [ ]:
$html        = $pivoted |   ConvertTo-Html -As Table -Fragment 
foreach   ( $rowHeading in $pivoted.$PivotBy) {
    $tree = $DataToPivot.where({$_.$pivotby -eq $rowHeading}) | Out-TreeView -ExcludeProperty $PivotBy -TitleHtml $rowHeading 
    $html = $html -replace "<td>\s*$rowHeading\s*</td>" ,"<td>$tree</td>"
}
Write-Notebook -Html $html

Day Average_Sales Min_Sales Max_Sales Sunday 
 
 location Product Sales 
 
 
 London Cucumber 28 
 London Tomato 31 
 London Lettuce 37 
 London Asparagus 29 
 London Potato 39 
 Edinburgh Cucumber 43 
 Edinburgh Tomato 42 
 Edinburgh Lettuce 35 
 Edinburgh Asparagus 37 
 Edinburgh Potato 28 
 Glasgow Cucumber 27 
 Glasgow Tomato 23 
 Glasgow Lettuce 29 
 Glasgow Asparagus 24 
 Glasgow Potato 24 
 Birmingham Cucumber 47 
 Birmingham Tomato 41 
 Birmingham Lettuce 42 
 Birmingham Asparagus 23 
 Birmingham Potato 29 
 Cardiff Cucumber 37 
 Cardiff Tomato 31 
 Cardiff Lettuce 21 
 Cardiff Asparagus 28 
 Cardiff Potato 26 
 
 
 34.267 21 47 Monday 
 
 location Product Sales 
 
 
 London Cucumber 46 
 London Tomato 35 
 London Lettuce 41 
 London Asparagus 49 
 London Potato 30 
 Edinburgh Cucumber 29 
 Edinburgh Tomato 26 
 Edinburgh Lettuce 36 
 Edinburgh Asparagus 25 
 Edinburgh Potato 35 
 Glasgow Cucumber 29 
 Glasgow Tomato 30 
 Glasgow Lettuce 25 
 Glasgow Asparagus 29 
 Glasgow Potato 47 
 Birmingham Cucumber 27 
 Birmingham Tomato 34 
 Birmingham Lettuce 49 
 Birmingham Asparagus 35 
 Birmingham Potato 31 
 Cardiff Cucumber 34 
 Cardiff Tomato 37 
 Cardiff Lettuce 40 
 Cardiff Asparagus 30 
 Cardiff Potato 41 
 
 
 34.533 25 49 Tuesday 
 
 location Product Sales 
 
 
 London Cucumber 30 
 London Tomato 26 
 London Lettuce 36 
 London Asparagus 38 
 London Potato 27 
 Edinburgh Cucumber 44 
 Edinburgh Tomato 48 
 Edinburgh Lettuce 32 
 Edinburgh Asparagus 26 
 Edinburgh Potato 28 
 Glasgow Cucumber 32 
 Glasgow Tomato 36 
 Glasgow Lettuce 46 
 Glasgow Asparagus 38 
 Glasgow Potato 22 
 Birmingham Cucumber 39 
 Birmingham Tomato 41 
 Birmingham Lettuce 41 
 Birmingham Asparagus 31 
 Birmingham Potato 20 
 Cardiff Cucumber 47 
 Cardiff Tomato 41 
 Cardiff Lettuce 26 
 Cardiff Asparagus 24 
 Cardiff Potato 49 
 
 
 37.667 26 48 Wednesday 
 
 location Product Sales 
 
 
 London Cucumber 25 
 London Tomato 26 
 London Lettuce 27 
 London Asparagus 31 
 London Potato 30 
 Edinburgh Cucumber 46 
 Edinburgh Tomato 43 
 Edinburgh Lettuce 38 
 Edinburgh Asparagus 41 
 Edinburgh Potato 26 
 Glasgow Cucumber 20 
 Glasgow Tomato 33 
 Glasgow Lettuce 26 
 Glasgow Asparagus 44 
 Glasgow Potato 27 
 Birmingham Cucumber 33 
 Birmingham Tomato 46 
 Birmingham Lettuce 28 
 Birmingham Asparagus 40 
 Birmingham Potato 47 
 Cardiff Cucumber 27 
 Cardiff Tomato 40 
 Cardiff Lettuce 38 
 Cardiff Asparagus 26 
 Cardiff Potato 32 
 
 
 33.067 20 46 Thursday 
 
 location Product Sales 
 
 
 London Cucumber 47 
 London Tomato 32 
 London Lettuce 44 
 London Asparagus 21 
 London Potato 37 
 Edinburgh Cucumber 39 
 Edinburgh Tomato 39 
 Edinburgh Lettuce 36 
 Edinburgh Asparagus 31 
 Edinburgh Potato 20 
 Glasgow Cucumber 21 
 Glasgow Tomato 29 
 Glasgow Lettuce 25 
 Glasgow Asparagus 35 
 Glasgow Potato 46 
 Birmingham Cucumber 21 
 Birmingham Tomato 46 
 Birmingham Lettuce 38 
 Birmingham Asparagus 29 
 Birmingham Potato 20 
 Cardiff Cucumber 29 
 Cardiff Tomato 28 
 Cardiff Lettuce 43 
 Cardiff Asparagus 26 
 Cardiff Potato 22 
 
 
 34.467 21 47 Friday 
 
 location Product Sales 
 
 
 London Cucumber 38 
 London Tomato 40 
 London Lettuce 35 
 London Asparagus 27 
 London Potato 39 
 Edinburgh Cucumber 36 
 Edinburgh Tomato 38 
 Edinburgh Lettuce 47 
 Edinburgh Asparagus 30 
 Edinburgh Potato 24 
 Glasgow Cucumber 39 
 Glasgow Tomato 36 
 Glasgow Lettuce 45 
 Glasgow Asparagus 28 
 Glasgow Potato 32 
 Birmingham Cucumber 31 
 Birmingham Tomato 32 
 Birmingham Lettuce 21 
 Birmingham Asparagus 32 
 Birmingham Potato 31 
 Cardiff Cucumber 21 
 Cardiff Tomato 43 
 Cardiff Lettuce 23 
 Cardiff Asparagus 37 
 Cardiff Potato 27 
 
 
 35 21 47 Saturday 
 
 location Product Sales 
 
 
 London Cucumber 32 
 London Tomato 29 
 London Lettuce 39 
 London Asparagus

## Two-way pivoting

In [ ]:
$PivotBy     = 'Product' 
$aggFunction = 'Average' 
$dataToPivot  |  New-PSPivotTable -index $PivotBy  -aggregateFunction $aggFunction -column day -values  sales  -outvar Pivoted -fill 0 | ft


Product   Average_Friday Average_Monday Average_Saturday Average_Sunday Average_Thursday Average_Tuesday Average_Wednesday
-------   -------------- -------------- ---------------- -------------- ---------------- --------------- -----------------
Asparagus            4.4            4.8            5.029          4.029            4.057           4.486               5.2
Cucumber           4.714          4.714            4.943            5.2            4.486           5.486             4.314
Lettuce            4.886          5.457            5.829          4.686            5.314           5.171             4.486
Potato             4.371          5.257                5          4.171            4.143           4.171             4.629
Tomato               5.4          4.629              4.4            4.8            4.971           5.486             5.371



## Format tricks


In [ ]:
$html        = $pivoted |   ConvertTo-Html -As Table -Fragment 
foreach   ( $rowHeading in $pivoted.$PivotBy) {
    $tree = $DataToPivot.where({$_.$pivotby -eq $rowHeading}) | Out-TreeView -ExcludeProperty $PivotBy -TitleHtml $rowHeading 
    $html = $html -replace "<td>\s*$rowHeading\s*</td>" ,"<td>$tree</td>"
}
Write-Notebook -Html $html

Product Average_Friday Average_Monday Average_Saturday Average_Sunday Average_Thursday Average_Tuesday Average_Wednesday Asparagus 
 
 Day location Sales 
 
 
 Monday London 49 
 Tuesday London 38 
 Wednesday London 31 
 Thursday London 21 
 Friday London 27 
 Saturday London 32 
 Sunday London 29 
 Monday Edinburgh 25 
 Tuesday Edinburgh 26 
 Wednesday Edinburgh 41 
 Thursday Edinburgh 31 
 Friday Edinburgh 30 
 Saturday Edinburgh 47 
 Sunday Edinburgh 37 
 Monday Glasgow 29 
 Tuesday Glasgow 38 
 Wednesday Glasgow 44 
 Thursday Glasgow 35 
 Friday Glasgow 28 
 Saturday Glasgow 33 
 Sunday Glasgow 24 
 Monday Birmingham 35 
 Tuesday Birmingham 31 
 Wednesday Birmingham 40 
 Thursday Birmingham 29 
 Friday Birmingham 32 
 Saturday Birmingham 26 
 Sunday Birmingham 23 
 Monday Cardiff 30 
 Tuesday Cardiff 24 
 Wednesday Cardiff 26 
 Thursday Cardiff 26 
 Friday Cardiff 37 
 Saturday Cardiff 38 
 Sunday Cardiff 28 
 
 
 4.4 4.8 5.029 4.029 4.057 4.486 5.2 Cucumber 
 
 Day location Sales 
 
 
 Monday London 46 
 Tuesday London 30 
 Wednesday London 25 
 Thursday London 47 
 Friday London 38 
 Saturday London 32 
 Sunday London 28 
 Monday Edinburgh 29 
 Tuesday Edinburgh 44 
 Wednesday Edinburgh 46 
 Thursday Edinburgh 39 
 Friday Edinburgh 36 
 Saturday Edinburgh 33 
 Sunday Edinburgh 43 
 Monday Glasgow 29 
 Tuesday Glasgow 32 
 Wednesday Glasgow 20 
 Thursday Glasgow 21 
 Friday Glasgow 39 
 Saturday Glasgow 22 
 Sunday Glasgow 27 
 Monday Birmingham 27 
 Tuesday Birmingham 39 
 Wednesday Birmingham 33 
 Thursday Birmingham 21 
 Friday Birmingham 31 
 Saturday Birmingham 44 
 Sunday Birmingham 47 
 Monday Cardiff 34 
 Tuesday Cardiff 47 
 Wednesday Cardiff 27 
 Thursday Cardiff 29 
 Friday Cardiff 21 
 Saturday Cardiff 42 
 Sunday Cardiff 37 
 
 
 4.714 4.714 4.943 5.2 4.486 5.486 4.314 Lettuce 
 
 Day location Sales 
 
 
 Monday London 41 
 Tuesday London 36 
 Wednesday London 27 
 Thursday London 44 
 Friday London 35 
 Saturday London 39 
 Sunday London 37 
 Monday Edinburgh 36 
 Tuesday Edinburgh 32 
 Wednesday Edinburgh 38 
 Thursday Edinburgh 36 
 Friday Edinburgh 47 
 Saturday Edinburgh 39 
 Sunday Edinburgh 35 
 Monday Glasgow 25 
 Tuesday Glasgow 46 
 Wednesday Glasgow 26 
 Thursday Glasgow 25 
 Friday Glasgow 45 
 Saturday Glasgow 33 
 Sunday Glasgow 29 
 Monday Birmingham 49 
 Tuesday Birmingham 41 
 Wednesday Birmingham 28 
 Thursday Birmingham 38 
 Friday Birmingham 21 
 Saturday Birmingham 46 
 Sunday Birmingham 42 
 Monday Cardiff 40 
 Tuesday Cardiff 26 
 Wednesday Cardiff 38 
 Thursday Cardiff 43 
 Friday Cardiff 23 
 Saturday Cardiff 47 
 Sunday Cardiff 21 
 
 
 4.886 5.457 5.829 4.686 5.314 5.171 4.486 Potato 
 
 Day location Sales 
 
 
 Monday London 30 
 Tuesday London 27 
 Wednesday London 30 
 Thursday London 37 
 Friday London 39 
 Saturday London 31 
 Sunday London 39 
 Monday Edinburgh 35 
 Tuesday Edinburgh 28 
 Wednesday Edinburgh 26 
 Thursday Edinburgh 20 
 Friday Edinburgh 24 
 Saturday Edinburgh 39 
 Sunday Edinburgh 28 
 Monday Glasgow 47 
 Tuesday Glasgow 22 
 Wednesday Glasgow 27 
 Thursday Glasgow 46 
 Friday Glasgow 32 
 Saturday Glasgow 29 
 Sunday Glasgow 24 
 Monday Birmingham 31 
 Tuesday Birmingham 20 
 Wednesday Birmingham 47 
 Thursday Birmingham 20 
 Friday Birmingham 31 
 Saturday Birmingham 29 
 Sunday Birmingham 29 
 Monday Cardiff 41 
 Tuesday Cardiff 49 
 Wednesday Cardiff 32 
 Thursday Cardiff 22 
 Friday Cardiff 27 
 Saturday Cardiff 47 
 Sunday Cardiff 26 
 
 
 4.371 5.257 5 4.171 4.143 4.171 4.629 Tomato 
 
 Day location Sales 
 
 
 Monday London 35 
 Tuesday London 26 
 Wednesday London 26 
 Thursday London 32 
 Friday London 40 
 Saturday London 29 
 Sunday London 31 
 Monday Edinburgh 26 
 Tuesday Edinburgh 48 
 Wednesday Edinburgh 43 
 Thursday Edinburgh 39 
 Friday Edinburgh 38 
 Saturda

In [ ]:
# With the data in $pivoted we can just do:  $pivoted | ConvertTo-Grid  -Display but this is interesting. 
New-Alias -Name pivot New-PSPivotTable -Force   # Aliases to make the next bit more of a DSL 
New-Alias -Name melt  Invoke-PSMelt -Force      #
$d = $data                                      # $data is defined inside Out-Cell so the script block will pick that up which isn't what we want
cell -AsGrid {
    $d |
     melt  Day, Location -ValueName Sales -VarName Product | 
     pivot Product Sales -Column Day -Fill 0
}  

Product 
 Average_Friday 
 Average_Monday 
 Average_Saturday 
 Average_Sunday 
 Average_Thursday 
 Average_Tuesday 
 Average_Wednesday 

 Asparagus 
 4.40 
 4.80 
 5.03 
 4.03 
 4.06 
 4.49 
 5.20 

 Cucumber 
 4.71 
 4.71 
 4.94 
 5.20 
 4.49 
 5.49 
 4.31 

 Lettuce 
 4.89 
 5.46 
 5.83 
 4.69 
 5.31 
 5.17 
 4.49 

 Potato 
 4.37 
 5.26 
 5.00 
 4.17 
 4.14 
 4.17 
 4.63 

 Tomato 
 5.40 
 4.63 
 4.40 
 4.80 
 4.97 
 5.49 
 5.37

In [ ]:
$PivotCol     = 'Day' 
$PivotRow     = 'Product'
$PivotGroupBy = 'Location'
$ValueName    = 'Sales' 
$aggFunction  = 'Average' 

$dataToPivot  =  $data        | Invoke-PSMelt    -Id    $PivotCol, $PivotGroupBy  -ValueName $ValueName -VarName $PivotRow
$pivoted      =  $dataToPivot | New-PSPivotTable -index $PivotRow -aggregateFunction $aggFunction -column $PivotCol -values $ValueName -Fill 0
$html         =  $pivoted     | ConvertTo-Grid  

foreach   ( $rowHeading in $pivoted.$PivotRow) {
    $branchRows = $dataToPivot.where( {$_.$PivotRow -eq $rowHeading} ) 
    $tree       = $branchRows.$PivotGroupBy | Sort-Object -Unique | Foreach-Object {
                  $leveltwoHeading = $_ 
                  $subgroup = $branchRows.where({$_.$PivotGroupBy -eq $leveltwoHeading})
                  [pscustomobject]@{$PivotGroupBy =  Out-TreeView $subgroup -ExcludeProperty $PivotRow,$PivotGroupBy -TitleHtml $leveltwoHeading} 
    } | Out-TreeView -Title $rowHeading  
    $html = $html -replace  "<(div class=`"grid-item`".*?)>\s*$rowheading\s*</div>", "<`$1>$tree</div>"
}
Write-Notebook $html

Product 
 Average_Friday 
 Average_Monday 
 Average_Saturday 
 Average_Sunday 
 Average_Thursday 
 Average_Tuesday 
 Average_Wednesday 

 Asparagus 
 
 Location 
 
 
 Birmingham 
 
 Day Sales 
 
 
 Monday 35 
 Tuesday 31 
 Wednesday 40 
 Thursday 29 
 Friday 32 
 Saturday 26 
 Sunday 23 
 
 
 
 Cardiff 
 
 Day Sales 
 
 
 Monday 30 
 Tuesday 24 
 Wednesday 26 
 Thursday 26 
 Friday 37 
 Saturday 38 
 Sunday 28 
 
 
 
 Edinburgh 
 
 Day Sales 
 
 
 Monday 25 
 Tuesday 26 
 Wednesday 41 
 Thursday 31 
 Friday 30 
 Saturday 47 
 Sunday 37 
 
 
 
 Glasgow 
 
 Day Sales 
 
 
 Monday 29 
 Tuesday 38 
 Wednesday 44 
 Thursday 35 
 Friday 28 
 Saturday 33 
 Sunday 24 
 
 
 
 London 
 
 Day Sales 
 
 
 Monday 49 
 Tuesday 38 
 Wednesday 31 
 Thursday 21 
 Friday 27 
 Saturday 32 
 Sunday 29 
 
 
 
 
 
 
 4.40 
 4.80 
 5.03 
 4.03 
 4.06 
 4.49 
 5.20 

 Cucumber 
 
 Location 
 
 
 Birmingham 
 
 Day Sales 
 
 
 Monday 27 
 Tuesday 39 
 Wednesday 33 
 Thursday 21 
 Friday 31 
 Saturday 44 
 Sunday 47 
 
 
 
 Cardiff 
 
 Day Sales 
 
 
 Monday 34 
 Tuesday 47 
 Wednesday 27 
 Thursday 29 
 Friday 21 
 Saturday 42 
 Sunday 37 
 
 
 
 Edinburgh 
 
 Day Sales 
 
 
 Monday 29 
 Tuesday 44 
 Wednesday 46 
 Thursday 39 
 Friday 36 
 Saturday 33 
 Sunday 43 
 
 
 
 Glasgow 
 
 Day Sales 
 
 
 Monday 29 
 Tuesday 32 
 Wednesday 20 
 Thursday 21 
 Friday 39 
 Saturday 22 
 Sunday 27 
 
 
 
 London 
 
 Day Sales 
 
 
 Monday 46 
 Tuesday 30 
 Wednesday 25 
 Thursday 47 
 Friday 38 
 Saturday 32 
 Sunday 28 
 
 
 
 
 
 
 4.71 
 4.71 
 4.94 
 5.20 
 4.49 
 5.49 
 4.31 

 Lettuce 
 
 Location 
 
 
 Birmingham 
 
 Day Sales 
 
 
 Monday 49 
 Tuesday 41 
 Wednesday 28 
 Thursday 38 
 Friday 21 
 Saturday 46 
 Sunday 42 
 
 
 
 Cardiff 
 
 Day Sales 
 
 
 Monday 40 
 Tuesday 26 
 Wednesday 38 
 Thursday 43 
 Friday 23 
 Saturday 47 
 Sunday 21 
 
 
 
 Edinburgh 
 
 Day Sales 
 
 
 Monday 36 
 Tuesday 32 
 Wednesday 38 
 Thursday 36 
 Friday 47 
 Saturday 39 
 Sunday 35 
 
 
 
 Glasgow 
 
 Day Sales 
 
 
 Monday 25 
 Tuesday 46 
 Wednesday 26 
 Thursday 25 
 Friday 45 
 Saturday 33 
 Sunday 29 
 
 
 
 London 
 
 Day Sales 
 
 
 Monday 41 
 Tuesday 36 
 Wednesday 27 
 Thursday 44 
 Friday 35 
 Saturday 39 
 Sunday 37 
 
 
 
 
 
 
 4.89 
 5.46 
 5.83 
 4.69 
 5.31 
 5.17 
 4.49 

 Potato 
 
 Location 
 
 
 Birmingham 
 
 Day Sales 
 
 
 Monday 31 
 Tuesday 20 
 Wednesday 47 
 Thursday 20 
 Friday 31 
 Saturday 29 
 Sunday 29 
 
 
 
 Cardiff 
 
 Day Sales 
 
 
 Monday 41 
 Tuesday 49 
 Wednesday 32 
 Thursday 22 
 Friday 27 
 Saturday 47 
 Sunday 26 
 
 
 
 Edinburgh 
 
 Day Sales 
 
 
 Monday 35 
 Tuesday 28 
 Wednesday 26 
 Thursday 20 
 Friday 24 
 Saturday 39 
 Sunday 28 
 
 
 
 Glasgow 
 
 Day Sales 
 
 
 Monday 47 
 Tuesday 22 
 Wednesday 27 
 Thursday 46 
 Friday 32 
 Saturday 29 
 Sunday 24 
 
 
 
 London 
 
 Day Sales 
 
 
 Monday 30 
 Tuesday 27 
 Wednesday 30 
 Thursday 37 
 Friday 39 
 Saturday 31 
 Sunday 39 
 
 
 
 
 
 
 4.37 
 5.26 
 5.00 
 4.17 
 4.14 
 4.17 
 4.63 

 Tomato 
 
 Location 
 
 
 Birmingham 
 
 Day Sales 
 
 
 Monday 34 
 Tuesday 41 
 Wednesday 46 
 Thursday 46 
 Friday 32 
 Saturday 20 
 Sunday 41 
 
 
 
 Cardiff 
 
 Day Sales 
 
 
 Monday 37 
 Tuesday 41 
 Wednesday 40 
 Thursday 28 
 Friday 43 
 Saturday 44 
 Sunday 31 
 
 
 
 Edinburgh 
 
 Day Sales 
 
 
 Monday 26 
 Tuesday 48 
 Wednesday 43 
 Thursday 39 
 Friday 38 
 Saturday 27 
 Sunday 42 
 
 
 
 Glasgow 
 
 Day Sales 
 
 
 Monday 30 
 Tuesday 36 
 Wednesday 33 
 Thursday 29 
 Friday 36 
 Saturday 34 
 Sunday 23 
 
 
 
 London 
 
 Day Sa

## SubTotals
Group by more than one column with more than one aggregate 


In [ ]:

$data | Invoke-PSMelt -Id Day, location -ValueName Sales -VarName Product -OutVariable DataToPivot  | Get-Subtotal -Sum -max -min -ValueName sales -GroupByName Location,Product  | ft -a | out-string


Location   Product   sales_Minimum sales_Maximum sales_Sum
--------   -------   ------------- ------------- ---------
Birmingham Asparagus            23            40       216
Birmingham Cucumber             21            47       242
Birmingham Lettuce              21            49       265
Birmingham Potato               20            47       207
Birmingham Tomato               20            46       260
Cardiff    Asparagus            24            38       209
Cardiff    Cucumber             21            47       237
Cardiff    Lettuce              21            47       238
Cardiff    Potato               22            49       244
Cardiff    Tomato               28            44       264
Edinburgh  Asparagus            25            47       237
Edinburgh  Cucumber             29            46       270
Edinburgh  Lettuce              32            47       263
Edinburgh  Potato               20            39       200
Edinburgh  Tomato               26            48       

## CrossTab  

Can do this with subtotals with one number column, but data doesn't need to be numeric 

In [ ]:
ConvertFrom-Csv @'
Environment,Service,IP
Dev,Apple,10.1.2.11
Test,Apple,10.2.3.11
Prod,Apple,10.3.4.11
Dev,Orange,10.1.2.21
Test,Orange,10.2.3.21
Prod,Orange,10.3.4.21
Dev,Banana,10.1.2.22
Test,Banana,10.2.3.22
Prod,Banana,10.3.4.22
'@   | ConvertTo-CrossTab -ColumnName Environment -RowName Service -ValueName Ip -SuffixColumn " environment"


Service Dev environment Prod environment Test environment
------- --------------- ---------------- ----------------
Apple   10.1.2.11       10.3.4.11        10.2.3.11
Banana  10.1.2.22       10.3.4.22        10.2.3.22
Orange  10.1.2.21       10.3.4.21        10.2.3.21

